Combining and cleaning datasets to create features

In [44]:
import pandas as pd
import numpy as np
import os

In [45]:
players = pd.read_csv("./data/players.csv")

In [46]:
players[:10]

,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,...,unit_order_glyph,unit_order_eject_item_from_stash,unit_order_cast_rune,unit_order_ping_ability,unit_order_move_to_direction,unit_order_patrol,unit_order_vector_target_position,unit_order_radar,unit_order_set_item_combine_lock,unit_order_continue
0,0,0,86,0,3261,10960,347,362,9,3,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,51,1,2954,17760,494,659,13,3,...,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,83,2,110,12195,350,385,0,4,...,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2,11,3,1179,22505,599,605,8,4,...,1.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,3,67,4,3307,23825,613,762,20,3,...,3.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN
5,0,4,106,128,476,12285,397,524,5,6,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
6,0,0,102,129,317,10355,303,369,4,13,...,4.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,5,46,130,2390,13395,452,517,4,8,...,NaN,NaN,NaN,4.0,110.0,NaN,NaN,NaN,NaN,NaN
8,0,0,7,131,475,5035,189,223,1,14,...,1.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
9,0,6,73,132,60,17550,496,456,1,11,...,1.0,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
match = pd.read_csv("./data/match.csv")

Merge match and players

In [48]:
playersmatch = pd.DataFrame(players.merge(match))

In [49]:
playersmatch[:10]

,match_id,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,...,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster
0,0,0,86,0,3261,10960,347,362,9,3,...,1982,4,3,63,1,22,True,0,1,155
1,0,1,51,1,2954,17760,494,659,13,3,...,1982,4,3,63,1,22,True,0,1,155
2,0,0,83,2,110,12195,350,385,0,4,...,1982,4,3,63,1,22,True,0,1,155
3,0,2,11,3,1179,22505,599,605,8,4,...,1982,4,3,63,1,22,True,0,1,155
4,0,3,67,4,3307,23825,613,762,20,3,...,1982,4,3,63,1,22,True,0,1,155
5,0,4,106,128,476,12285,397,524,5,6,...,1982,4,3,63,1,22,True,0,1,155
6,0,0,102,129,317,10355,303,369,4,13,...,1982,4,3,63,1,22,True,0,1,155
7,0,5,46,130,2390,13395,452,517,4,8,...,1982,4,3,63,1,22,True,0,1,155
8,0,0,7,131,475,5035,189,223,1,14,...,1982,4,3,63,1,22,True,0,1,155
9,0,6,73,132,60,17550,496,456,1,11,...,1982,4,3,63,1,22,True,0,1,155


In [50]:
# Player is on winning team if one of the first two following conditions are true
winorlose_conditions = [
    (playersmatch['radiant_win'] == True) & (playersmatch['player_slot'] < 5),
    (playersmatch['radiant_win'] == False) & (playersmatch['player_slot'] > 4),
    (playersmatch['radiant_win'] == True) & (playersmatch['player_slot'] > 4),
    (playersmatch['radiant_win'] == False) & (playersmatch['player_slot'] < 5)
]

# Winners given a 1 flag, losers given a 0 flag
winorlose = [1, 1, 0, 0]

In [51]:
# Assign winning flag to players based on condition 
playersmatch['win'] = np.select(winorlose_conditions, winorlose)

In [52]:
# Match hero names to match IDs
heroes = pd.read_csv("./data/hero_names.csv")
playersmatch = pd.DataFrame(heroes.merge(playersmatch))

In [57]:
playersmatch = playersmatch.sort_values(by = ['match_id', 'player_slot'])
playersmatch[:50]

,name,hero_id,localized_name,match_id,account_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,...,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster,win
402221,npc_dota_hero_rubick,86,Rubick,0,0,0,3261,10960,347,362,...,4,3,63,1,22,True,0,1,155,1
265414,npc_dota_hero_rattletrap,51,Clockwerk,0,1,1,2954,17760,494,659,...,4,3,63,1,22,True,0,1,155,1
390175,npc_dota_hero_treant,83,Treant Protector,0,0,2,110,12195,350,385,...,4,3,63,1,22,True,0,1,155,1
60411,npc_dota_hero_nevermore,11,Shadow Fiend,0,2,3,1179,22505,599,605,...,4,3,63,1,22,True,0,1,155,1
311776,npc_dota_hero_spectre,67,Spectre,0,3,4,3307,23825,613,762,...,4,3,63,1,22,True,0,1,155,1
476336,npc_dota_hero_ember_spirit,106,Ember Spirit,0,4,128,476,12285,397,524,...,4,3,63,1,22,True,0,1,155,0
461927,npc_dota_hero_abaddon,102,Abaddon,0,0,129,317,10355,303,369,...,4,3,63,1,22,True,0,1,155,0
242981,npc_dota_hero_templar_assassin,46,Templar Assassin,0,5,130,2390,13395,452,517,...,4,3,63,1,22,True,0,1,155,0
29960,npc_dota_hero_earthshaker,7,Earthshaker,0,0,131,475,5035,189,223,...,4,3,63,1,22,True,0,1,155,0
351596,npc_dota_hero_alchemist,73,Alchemist,0,6,132,60,17550,496,456,...,4,3,63,1,22,True,0,1,155,0


In [59]:
playersmatch.to_csv("./data/combined.csv")